In [ ]:
# import os
# import pandas as pd

# # Local cache directory and file path
# cache_dir = '/home/sagemaker-user/data'
# os.makedirs(cache_dir, exist_ok=True)
# local_parquet_path = os.path.join(cache_dir, 'med_xgb_data.parquet')

# # Your existing CSV S3 path
# s3_csv_path = 's3://mlops-sandbox-bucket/med_xgb_data.csv'

# # Parquet S3 path (same bucket/folder but different filename)
# s3_parquet_path = 's3://mlops-sandbox-bucket/med_xgb_data.parquet'

# if os.path.exists(local_parquet_path):
#     # Load from local cache if exists
#     df = pd.read_parquet(local_parquet_path)
#     print("Loaded data from local parquet cache")
# else:
#     # First time loading: read CSV from S3 (slow)
#     df = pd.read_csv(s3_csv_path)
#     print("Loaded data from CSV on S3")

#     # Save parquet locally for future quick loads
#     df.to_parquet(local_parquet_path)
#     print(f"Saved parquet locally at {local_parquet_path}")

#     # Also save parquet back to S3 for persistence/sharing
#     df.to_parquet(s3_parquet_path, index=False)
#     print(f"Saved parquet to S3 at {s3_parquet_path}")


In [ ]:
# Use the Conversation API to send a text message to Amazon Nova.

import boto3
from botocore.exceptions import ClientError

# Create a Bedrock Runtime client in the AWS Region you want to use.
client = boto3.client("bedrock-runtime", region_name="us-east-2")

# Set the model ID, e.g., Amazon Nova Lite.
model_id = "amazon.nova-lite-v1:0"

# Start a conversation with the user message.
user_message = "Describe the purpose of a 'hello world' program in one line."
conversation = [
    {
        "role": "user",
        "content": [{"text": user_message}],
    }
]

try:
    # Send the message to the model, using a basic inference configuration.
    response = client.converse(
        modelId=model_id,
        messages=conversation,
        inferenceConfig={"maxTokens": 512, "temperature": 0.5, "topP": 0.9},
    )

    # Extract and print the response text.
    response_text = response["output"]["message"]["content"][0]["text"]
    print(response_text)

except (ClientError, Exception) as e:
    print(f"ERROR: Can't invoke '{model_id}'. Reason: {e}")
    exit(1)




In [1]:
# Use the Conversation API to send a text message to Amazon Nova.

import boto3
from botocore.exceptions import ClientError

# Create a Bedrock Runtime client in the AWS Region you want to use.
client = boto3.client("bedrock-runtime", region_name="us-east-2")

# Set the model ID, e.g., Amazon Nova Lite.
model_id = "amazon.nova-lite-v1:0"

# Start a conversation with the user message.
user_message = "Describe the purpose of a 'hello world' program in one line."
conversation = [
    {
        "role": "user",
        "content": [{"text": user_message}],
    }
]

try:
    # Send the message to the model, using a basic inference configuration.
    response = client.converse(
        modelId=model_id,
        messages=conversation,
        inferenceConfig={"maxTokens": 512, "temperature": 0.5, "topP": 0.9},
    )

    # Extract and print the response text.
    response_text = response["output"]["message"]["content"][0]["text"]
    print(response_text)

except (ClientError, Exception) as e:
    print(f"ERROR: Can't invoke '{model_id}'. Reason: {e}")
    exit(1)




ERROR: Can't invoke 'amazon.nova-lite-v1:0'. Reason: An error occurred (AccessDeniedException) when calling the Converse operation: User: arn:aws:sts::168264144360:assumed-role/SageMaker-ExecutionRole-20250604T230532/SageMaker is not authorized to perform: bedrock:InvokeModel on resource: arn:aws:bedrock:us-east-2::foundation-model/amazon.nova-lite-v1:0 because no identity-based policy allows the bedrock:InvokeModel action


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, log_loss, confusion_matrix, ConfusionMatrixDisplay
)
import matplotlib.pyplot as plt
import seaborn as sns
import mlflow
import mlflow.xgboost

In [ ]:
mlflow.set_tracking_uri("arn:aws:sagemaker:us-east-2:168264144360:mlflow-tracking-server/mlflow-tracker")

In [ ]:
df = pd.read_parquet('/home/sagemaker-user/data/med_xgb_data.parquet')

In [ ]:
df

In [ ]:
target_col = 'target' 
X = df.drop(columns=[target_col])
y = df[target_col]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
# Train the model (no MLflow here)
model = xgb.XGBClassifier(
    n_estimators=50,
    max_depth=3,
    learning_rate=0.1,
    subsample=0.7,
    colsample_bytree=0.7,
    tree_method='hist',
    eval_metric='logloss',
    random_state=42
)
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)[:, 1]


In [ ]:
mlflow.set_experiment("test_experiment")

with mlflow.start_run(run_name="param_test"):
    mlflow.log_param("n_estimators", 50)
    print("✅ Logged single parameter to MLflow")

In [ ]:
mlflow.set_experiment("xgb_experiment")

with mlflow.start_run(run_name="post_training_logging"):
    for k, v in metrics.items():
        mlflow.log_metric(k, v)
        
    # mlflow.xgboost.log_model(model, artifact_path="model")

    print("✅ Metrics and model logged post training.")


In [ ]:
import os
import mlflow
# from mlflow.tracking import MlflowClient
# local_tracking_dir = "s3://mlops-sandbox-bucket/mlflow_tracker/"
# mlflow.set_tracking_uri(f"file://{local_tracking_dir}")

In [ ]:
# !pip show sagemaker-mlflow mlflow

In [ ]:


# # Initialize MLflow client
# client = MlflowClient()

# # List experiments and their runs
# experiments = client.list_experiments()
# for exp in experiments:
#     print(f"\nExperiment: {exp.name}")
#     print(f"  ID: {exp.experiment_id}")
#     print(f"  Artifact Location: {exp.artifact_location}")

#     runs = client.search_runs([exp.experiment_id])
#     for run in runs:
#         print(f"    Run ID: {run.info.run_id}")
#         print(f"    Metrics: {run.data.metrics}")
#         print(f"    Params: {run.data.params}")


In [ ]:
import mlflow
import sagemaker_mlflow

mlflow.set_tracking_uri("arn:aws:sagemaker:<region>:<account>:mlflow-tracking-server/<name>")
mlflow.set_experiment("test-exp")
with mlflow.start_run():
    mlflow.log_metric("accuracy", 0.93)


In [ ]:
import mlflow
import sagemaker_mlflow

In [ ]:
mlflow.set_tracking_uri("arn:aws:sagemaker:us-east-2:168264144360:mlflow-tracking-server/mlflow-tracker")

In [ ]:
print(mlflow.tracking.get_tracking_uri())

In [ ]:
import logging
logging.basicConfig(level=logging.DEBUG)

In [ ]:
from mlflow.tracking import _tracking_service
print(_tracking_service.utils._tracking_store_registry._registry)


In [ ]:
server_arn = 'arn:aws:sagemaker:us-east-2:168264144360:mlflow-tracking-server/mlflow-tracker'

In [ ]:
mlflow.set_tracking_uri(server_arn)

In [ ]:
with mlflow.start_run():
    mlflow.log_metric("foo", 1)
    
print(mlflow.search_runs())

In [ ]:
mlflow.set_experiment("simple_test")

# Start the run and log data
with mlflow.start_run():
    mlflow.log_param("param1", 5)
    mlflow.log_metric("accuracy", 0.91)
    mlflow.log_metric("loss", 0.08)

print("✅ Logged successfully to:", run.info.artifact_uri)